# 1. Short Questions
---

*1.1) Give an example of “omitted variable bias”*

=>**Omitted variable bias arises when the regressor ‘X’ is correlated with an omitted variable/unobservable variable. This results in an incorrectly specified model. For example, if we are running a regression between consumption as regressand and income as the regressor. An omitted variable bias (OVB) can occur when the model excludes one or more relevant variables such as inflation (macroeconomic factor) or other fixed expenditures as parameters. This results in an unfair value of beta. Thus, OVB affects the regression's explanatory power**


*1.2) Suppose you are interested in the effect of being included in the S&P 500 index on stock prices. Give at least two reasons why OLS might not be the best linear unbiased estimator*

=>**Although, the broad assumptions of OLS include that the model should be linear and that the sample should consist of identical, independent random variables. OLS might not be BLUE since the following conditions are NOT met:** 
> **Homoskedasticity(sigma square = 0)**

> **Uncorrelated shocks(ui,uj =0)**

=>**The effect of being included in the S&P 500 on stock prices should not be modeled using OLS. Not only because stock prices are a highly volatile dataset, but also since the variance of the error term is not constant. Further, shocks in the past can have an effect on the current market price. Moreover, stock prices reflect customer sentiment and their general perception towards markets being bullish or bearish**

*1.3) [True/False/Uncertain] If fixed effects model and random effects model give very different estimates, then the estimate from the random-effects model must be inconsistent*

=>**It depends. Based on the assumptions. The fixed-effects model rests on the assumption that the individual effects are correlated with the regressors. The underlying principle being that the unobserved characteristics of a firm are fixed. Based on this they remove the biases of the model by transforming the data. The random-effects model, on the other hand, assumes that the individual effects are uncorrelated**

**This essentially means that random effects are efficient. They should be used in addition to the fixed effects provided their assumptions hold true**

*1.4. There are four biosafety levels for viruses and other biological agents (BSL-1 - BSL-4), ranked by the level of precautions required in lab facility (For example, HIV is level 2, SARS is level 3, and Ebola is level 4). If we want to use the characteristics of the new coronavirus to predict which level it will be at, which model should we use?*

=>**Logit Model using dummy variables**


 

# 2. Housing Prices

---



## Loading data

In this exercise you will work with a dataset containing housing prices from a large sample of neighborhoods.

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

First we load data on housing prices and neighborhood characteristics.

In [0]:
data1 = pd.read_excel('https://drive.google.com/uc?export=download&id=1j6j_z_hJllt27ol9Jw1xhOCm8GMqD7Nk')
data1.describe()

## Variable List

- price: Median housing price

- crime: Crimes per capita

- nox: Nitrous oxide in parts per 100 million

- rooms: Average number of rooms

- dist: Weighted distance to employment centers

- radial: Accessibility index to radial highways

- proptax: Property tax per $1,000


**1. Run a regression with the following model, and report the $R^{2}$ of the regression.**

$$price_{i}=\beta_{0}+\beta_{1}crime_{i}+\beta_{2}nox_{i}+\beta_{3}dist_{i}+\beta_{4}radial_{i}+\beta_{5}proptax_{i}+\epsilon_{i}$$

In [0]:
formula1 = 'price ~ crime + nox + dist + radial + proptax'
result1 = smf.ols(formula1, data1).fit()
print(result1.summary())

**What is the $R^2$?**

In [0]:
print("R-squared:",round(result1.rsquared,4))

**2. Run a Breusch-Pagan test for heteroskedasticity**

In [0]:
from statsmodels.compat import lzip

name = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
test = sms.het_breuschpagan(result1.resid, result1.model.exog)
lzip(name, test)

In [0]:
print("NULL HYPOTHESIS (H0): The error variances are all equal ")
print("ALTHERNATIVE (Ha): The error variances are a multiplicative function of one or more variables")
print("Breusch Pagan Test");print("p value = ",round(test[1],4))
if(test[1]>0.05):
  print("FAIL TO REJECT NULL i.e Reject Alternative") 
else: 
  print("REJECT NULL HYPOTHESIS i.e Accept Alternative")

**Is there heteroskedasticity present?**

Answer: Yes, heteroskedasticity is present

**If there is heteroskedasticity, fix this problem by running a second regression using White standard errors.**




In [0]:
formula2 = 'price ~ crime + nox + dist + radial + proptax'
result2 = smf.ols(formula2, data).fit(cov_type='HC1')
print(result2.summary())

**Explain what has (and hasn’t) changed.**

Answer: The value of F-statistic has changed marginally i.e. 2*10^45. The F-statistic measures the joint effect of all variables combined 

In [0]:
#print("Residuals with standard OLS")
plt.plot(result1.resid)

In [0]:
#print("Residuals with White Standard Error")
plt.plot(result2.resid)

No considerable difference between the residuals of OLS and White standard Error test 

**3. Run an F-test in Stata to check if *proptax* and *radial* jointly have no effect.**

In [0]:
hypothesis = "proptax = radial"
f_test=result2.f_test(hypothesis)

In [0]:
print("F-test for joint hypothesis Test")
print(f_test)

**What do you conclude?**

Answer: A small p-value (typically ≤ 0.05) indicates strong evidence against the null hypothesis, so we reject the null hypothesis which states that proptax and radial are jointly significant 

**4. Run a Ramsey RESET test for functional form specification.**

In [0]:
from statsmodels.stats.outliers_influence import reset_ramsey
formula3 = 'price ~ crime + nox + dist + radial + proptax'
result3 = smf.ols(formula3, data1).fit()
reset_ramsey(result3, degree=5)

**Is the linear model misspecified?**

Answer: With an F-value of 443.3566 and a corresponding p-value of 0.09e-110, the RESET test results suggest that we reject the null hypothesis which states that the model has no omitted variables. In other words, we find strong evidence that the chosen linear functional form of the model is correct.

**5. Run a regression using a log variant of the original model, as shown below. [Hint: You will need to create a new variable that is the natural log of price.] Be sure to use White standard errors if you found there was heteroskedasticity.**

$$\log(price_{i})=\beta_{0}+\beta_{1}crime_{i}+\beta_{2}nox_{i}+\beta_{3}dist_{i}+\beta_{4}radial_{i}+\beta_{5}proptax_{i}+\epsilon_{i}$$

In [0]:
data1['log_price'] = np.log(data1['price'])
formula4 = 'log_price ~ crime + nox + dist + radial + proptax'
result4 = smf.ols(formula4, data1).fit()
print(result4.summary())

**Compare the goodness-of-fit statistics for this model and the original model. Which one is a better fit?**

In [0]:
print("Goodness of fit for model 1: ",round(result1.rsquared,4))
print("Goodness of fit for model 4: ",round(result4.rsquared,4))
if(result1.rsquared>result4.rsquared):
  print("Model 1 is better")
else:
  print("Model 4 is better")

# 3. Value of Patents

---



In this exercise we study the value of patents for firms. Suppose we have a panel dataset for a sample of publicly listed firms. For each firm we have the number of patents granted to that firm in each year and the stock price and market value by the end of each year.

*3.1) Consider an OLS model: where y_it is the market value, and p_it is the number of patents for firm i in year t. Suppose i is unobserved firm-specific characteristic, like firm culture and managerial talent. What assumption do you need for the OLS estimator to be unbiased? Do you think the assumption is satisfied here and why?*

**The assumption needed for the OLS estimator to be unbiased is that firm-specific characteristics (firm culture and managerial talent) are UNCORRELATED with variables that change over time i.e number of patents**

*3.2) If the assumption from 1 is satisfied, is the OLS estimator BLUE? What procedure can you do to get an estimator that is BLUE?*

**Even though the assumption from 3.1 is satisfied. It does not necessarily mean that the estimator is BLUE. Since it can still be inefficient i.e non-optimal solution set generated. To correct this, a transformation is required**

*3.3) If the assumption from 1 is not satisfied, what are the two ways to get unbiased and consistent estimates? Use one or two sentences to describe each approach*

**First difference: This procedure involves subtracting the previous value from the current value. This is called a first-order difference if lag is one period**

**Fixed Effects: The modeling approach includes the error term causing the bias in the regression itself**

*3.4) Now let’s consider how much R&D investment can produce a patent. When you
regress number of patents on R&D investment, which model should you use?*

**First Difference**


In [0]:
!pip install -q linearmodels
from linearmodels.panel import PooledOLS
from linearmodels.panel import PanelOLS as fe

## Loading data

In this exercise you will work with a dataset that matches the Compustat data to patents.

In [0]:
data2 = pd.read_excel('https://drive.google.com/uc?export=download&id=1NJWomov6NFJoMjvFZyuGzE2Qjc0IkDqQ')
## could take some time to load
data2.describe()

In [0]:
#Fill missing variables
idx = data2.loc[ (data2['npats'].isnull()) | (data2['npats']<=0) | (data2['npats'].isna())].index
data2.loc[idx, 'npats'] = 0

# get the index of rows where 'xrd' is negative or the value is missed
idx = data2.loc[ (data2['xrd'].isnull()) | (data2['xrd']<0)].index
data2.loc[idx, 'xrd'] = 0

# get the index of rows where 'csho' is negative or the value is missed
idx = data2.loc[ (data2['csho'].isnull()) | (data2['csho']<=0) | (data2['csho'].isna())].index
data2.loc[idx, 'csho'] = 53.298925 #Mean/Average (no. of shares outstanding 'csho', excluding NA values)


idx = data2.loc[ (data2['prcc_c'].isnull()) | (data2['prcc_c']<=0)| (data2['prcc_c'].isna())].index
# replace value of 'xrd' at those rows by zero.
data2.loc[idx, 'prcc_c'] = 22.995556 #Mean/Average (price per share 'prcc_c', excluding NA values)

#As data points become closer to the mean, data is more normally distributed
data2.info()

## Variable List

- permno: Company ID

- year: Year of observation

- npats: Number of patents the firm is granted in each year

- xrd: R&D investment (millions)

- csho: Number of common shares outstanding (millions)

- prcc_c: Year-end stock price

- conm: Company name


**5. How many firms have at least one patent? How many firms have no patent?**

In [0]:
temp1=data2.groupby('conm')['npats'].count()
print("Companies with atleast 1 patent:",temp1[temp1>=1].sum())

In [0]:
print("Companies with no patents:",temp1[temp1==0].sum())

**6. Calculate market capitalization by multiplying the stock price by shares outstanding. What is the average market capitalization?**

In [0]:
temp2_1=data2.groupby('conm')['prcc_c'].mean()
temp2_2=data2.groupby('conm')['csho'].mean()
market_cap = temp2_1*temp2_2
print(market_cap)

The average market capitalization is 

In [0]:
print("The average market capitalization is:",round(np.mean(market_cap),4))

**7. Create a variable “log_cap” that is the log market capitalization. Run an OLS regression of log market capitalization on the number of patents.**

In [0]:
data2.info()
data2['log_cap']=np.log(data2['prcc_c'])+  np.log(data2['csho'])

In [0]:
#Performing regression
formula5 = 'log_cap~npats'
result5 = smf.ols(formula5,data2).fit()
print(result5.summary())

The coefficient is 

In [0]:
print("The coefficient is:",round(result5.params[1],4))

**8. Run a regression of log market capitalization on the number of patents with firm and year fixed effects.**

In [134]:
exog_vars = ['npats']
exog = sm.add_constant(data2[exog_vars])

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [0]:
data2.reset_index(inplace=True)
data2.set_index(['conm', 'year'], inplace=True)
data2['log_cap1'] = data2.groupby('conm')['log_cap'].shift()

In [136]:
#Pooled OLS
mod = PooledOLS(data2['log_cap1'], exog)
pooled_res = mod.fit()
print(pooled_res)

/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PooledOLS Estimation Summary                          
Dep. Variable:               log_cap1   R-squared:                        0.0593
Estimator:                  PooledOLS   R-squared (Between):              0.0255
No. Observations:              195992   R-squared (Within):              -0.0288
Date:                Tue, Feb 11 2020   R-squared (Overall):              0.0593
Time:                        13:39:13   Log-likelihood                -4.265e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                   1.236e+04
Entities:                       19899   P-value                           0.0000
Avg Obs:                       9.8493   Distribution:                F(1,195990)
Min Obs:                       1.0000                                           
Max Obs:                       30.000   F-statistic (robust):          1.236e+04
                            

In [137]:
#With Year Fixed effects
mod = fe(data2['log_cap1'], exog, time_effects=True)
fe_year = mod.fit()
print(fe_year)

/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:               log_cap1   R-squared:                        0.0620
Estimator:                   PanelOLS   R-squared (Between):              0.0245
No. Observations:              195992   R-squared (Within):              -0.0250
Date:                Tue, Feb 11 2020   R-squared (Overall):              0.0592
Time:                        13:39:21   Log-likelihood                -4.141e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                   1.296e+04
Entities:                       19899   P-value                           0.0000
Avg Obs:                       9.8493   Distribution:                F(1,195960)
Min Obs:                       1.0000                                           
Max Obs:                       30.000   F-statistic (robust):          1.296e+04
                            

In [138]:
#With Firm Fixed Effects
mod = fe(data2['log_cap1'], exog,  entity_effects=True)
fe_firm = mod.fit()
print(fe_firm)

/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:               log_cap1   R-squared:                        0.0067
Estimator:                   PanelOLS   R-squared (Between):             -0.0084
No. Observations:              195992   R-squared (Within):               0.0067
Date:                Tue, Feb 11 2020   R-squared (Overall):              0.0305
Time:                        13:39:32   Log-likelihood                -2.834e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      1184.3
Entities:                       19899   P-value                           0.0000
Avg Obs:                       9.8493   Distribution:                F(1,176092)
Min Obs:                       1.0000                                           
Max Obs:                       30.000   F-statistic (robust):             1184.3
                            

In [139]:
#With Firm and Year Fixed Effects
mod = fe(data2['log_cap1'], exog,  entity_effects=True, time_effects=True)
fe_firm_year = mod.fit()
print(fe_firm_year)

/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:               log_cap1   R-squared:                        0.0031
Estimator:                   PanelOLS   R-squared (Between):             -0.0169
No. Observations:              195992   R-squared (Within):               0.0057
Date:                Tue, Feb 11 2020   R-squared (Overall):              0.0202
Time:                        13:39:40   Log-likelihood                -2.663e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      539.86
Entities:                       19899   P-value                           0.0000
Avg Obs:                       9.8493   Distribution:                F(1,176062)
Min Obs:                       1.0000                                           
Max Obs:                       30.000   F-statistic (robust):             539.86
                            

The coefficient is 

In [140]:
print("Coefficient using Pooled ols:",round(pooled_res.params[1],5))
print("Coefficient using Year fixed effect ols:",round(fe_year.params[1],5))
print("Coefficient using Firm  fixed effect ols:",round(fe_firm.params[1],5))
print("Coefficient using Firm and Year fixed effect ols:",round(fe_firm_year.params[1],5))


Coefficient using Pooled ols: 0.0326
Coefficient using Year fixed effect ols: 0.03135
Coefficient using Firm  fixed effect ols: 0.00986
Coefficient using Firm and Year fixed effect ols: 0.00613


**9. Run a first difference regression of log market capitalization on the number of patents.**

In [141]:
mod_firstdiff = data2.groupby('conm')[['log_cap1'] + exog_vars ].diff()
exog = sm.add_constant(mod_firstdiff[exog_vars])
mod = PooledOLS(mod_firstdiff['log_cap1'], exog)
firstdiff = mod.fit()
print(firstdiff)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PooledOLS Estimation Summary                          
Dep. Variable:               log_cap1   R-squared:                     8.779e-05
Estimator:                  PooledOLS   R-squared (Between):             -0.0165
No. Observations:              176093   R-squared (Within):           -1.079e-05
Date:                Tue, Feb 11 2020   R-squared (Overall):           8.779e-05
Time:                        13:40:14   Log-likelihood                -2.363e+05
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      15.461
Entities:                       19840   P-value                           0.0001
Avg Obs:                       8.8757   Distribution:                F(1,176091)
Min Obs:                       1.0000                                           
Max Obs:                       29.000   F-statistic (robust):             15.461
                            

The coefficient is 

In [142]:
print("Coefficient using First Difference ols:",round(firstdiff.params[1],5))

Coefficient using First Difference ols: 0.00183


**10. Using the average firm's market capitalization, what is the value of a patent?**

Answer: Using OLS, the value of a patent is

Using FE, the value of a patent is

Using FD, the value of a patent is

In [143]:
res = fe_firm_year.params
res.name = 'FE_firm_year'
res = res.to_frame() 
res['First_difference']= firstdiff.params 
res

,FE_firm_year,First_difference
const,4.779475,-0.004237
npats,0.006131,0.001826


In [144]:
(np.log(np.mean(market_cap)) - fe_firm_year.params[0])/fe_firm_year.params[1]

359.4070942814854

In [145]:
(np.log(np.mean(market_cap)) - firstdiff.params[0])/firstdiff.params[1]

3827.4889727537497

**11. Let's consider firms with only one patent (in all of the years), for which the patent could be more valuable.**

In [146]:
#Restrict the sample to firms with single patent
data3 = data2.loc[(data2['npats']==1)]
data3.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10490 entries, (CABOT MEDICAL CORP, 1990) to (YUKON ENERGY CORP, 1989)
Data columns (total 8 columns):
index       10490 non-null int64
permno      10490 non-null int64
npats       10490 non-null int64
xrd         10490 non-null float64
csho        10490 non-null float64
prcc_c      10490 non-null float64
log_cap     10490 non-null float64
log_cap1    10189 non-null float64
dtypes: float64(5), int64(3)
memory usage: 842.5+ KB


In [147]:
#FE
exog = sm.add_constant(data3[exog_vars])
mod = fe(data3['log_cap'], exog, entity_effects=True)
fe_firm = mod.fit()
print(fe_firm)

                          PanelOLS Estimation Summary                           
Dep. Variable:                log_cap   R-squared:                        0.0000
Estimator:                   PanelOLS   R-squared (Between):              0.0000
No. Observations:               10490   R-squared (Within):               0.0000
Date:                Tue, Feb 11 2020   R-squared (Overall):              0.0000
Time:                        13:40:46   Log-likelihood                -1.067e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                          --
Entities:                       19935   P-value                               --
Avg Obs:                       0.5262   Distribution:                         --
Min Obs:                       0.0000                                           
Max Obs:                       17.000   F-statistic (robust):                 --
                            

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [148]:
#FD
data3 = data2.loc[(data2['npats']==1)]
exog_vars = ['npats']
mod_firstdiff = data3.groupby('conm')[['log_cap1'] + exog_vars ].diff()
exog = sm.add_constant(data23[exog_vars])
mod = PooledOLS(mod_firstdiff['log_cap1'], exog)
firstdiff = mod.fit()
print(firstdiff)

                          PooledOLS Estimation Summary                          
Dep. Variable:               log_cap1   R-squared:                        0.0000
Estimator:                  PooledOLS   R-squared (Between):              0.0000
No. Observations:                5970   R-squared (Within):               0.0000
Date:                Tue, Feb 11 2020   R-squared (Overall):              0.0000
Time:                        13:40:56   Log-likelihood                   -8331.5
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                          --
Entities:                        2431   P-value                               --
Avg Obs:                       2.4558   Distribution:                         --
Min Obs:                       1.0000                                           
Max Obs:                       16.000   F-statistic (robust):                 --
                            

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


Answer: For the group of firms with a single patent,

Using FE, the value of a patent is: 4.8293 

Using FD, the value of a patent is: 0.1668

# 4. Predicting Sovereign Default

## Loading data

In this exercise you will work with a dataset from World Bank from 1991 to 2017 to predict default of sovereign debt. 

In [0]:
data4 = pd.read_csv('https://drive.google.com/uc?export=download&id=1-SVgD876f1uq69814leAAU6lXoFRdX7K')
data4.describe()

In [0]:
data4.info()

## Variable List

- Country: country name
- Year: year of observation
- default_RR: whether default (=1 if default)
- Netforeignassets_currentLCU : Net Foreign Assets (current LCU)
- Inflationconsumerprices_annualpc : Inflation (annual % change in CPI)
- Externalbalanceongoodsandservice : External Trade Balance (% of GDP)
- Currentaccountbalance_BoPcurrent : Current Account Balance (current USD)
- Nettradeingoodsandservices_BoPcu : Balance of Payments (current USD)
- Unemploymenttotal_pctoftotallabo : Unemployment (% of labor force)
- DGDP : Real GDP Growth (YOY % change)
- RYPC : Real GDP per capita growth (annual % change)
- CGEB : Change in Net Exports (% of GDP)
- PSBR : Central Government Balance (% of GDP)
- BINT : Interest Payments on Government Debt (% of GDP)
- PUDP : Total Debt owed by Government to Domestic and Foreign Creditors (% of GDP)
- SODD : Bank Lending to Public and Private Sectors (annual % change)
- CARA : Current Account Balance (% of GDP)
- IRTD : Total International Reserves (% of external debt stock)
- TDPX : Total External Debt Stock (% of exports)
- TDPY : Total External Debt (% of GDP)
- TSPY : Total External Debt Service Paid (% of GDP)
- INPS : Total Interest Payments made on External Debt (% of total debt service paid)
- INPY : Total Interest Payments made on External Debt (% of GDP)
- XRRE : Real Effective Exchange Rate (weighted by trade)


In [0]:
from statsmodels.formula.api import logit
from statsmodels.formula.api import probit
from statsmodels.formula.api import ols
from sklearn.utils import resample 
import scipy.stats
import sklearn.metrics as sklm

**1. Split the data into a training sample (before 2010) and a test sample (2010 and afterwards).**

In [0]:
train = data4.loc[(data4['Year']<2010)]
test =  data4.loc[(data4['Year']>=2010)]

**Fit an OLS model to the training sample and predict the default in the test sample.**

In [194]:
formula = 'default_RR ~ Netforeignassets_currentLCU + Inflationconsumerprices_annualpc + Externalbalanceongoodsandservice + Currentaccountbalance_BoPcurrent + Nettradeingoodsandservices_BoPcu + Unemploymenttotal_pctoftotallabo + DGDP + RYPC + CGEB + PSBR + BINT + PUDP + SODD + CARA + IRTD + TDPX + TDPY + TSPY + INPS + INPY + XRRE'
results_ols = ols(formula, test).fit()
print(results_ols.summary())

                            OLS Regression Results                            
Dep. Variable:             default_RR   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 7.000e+13
Date:                Tue, 11 Feb 2020   Prob (F-statistic):               0.00
Time:                        15:13:17   Log-Likelihood:                 8175.1
No. Observations:                 496   AIC:                        -1.631e+04
Df Residuals:                     476   BIC:                        -1.623e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

In [195]:
exog_var=['Netforeignassets_currentLCU','Inflationconsumerprices_annualpc','Externalbalanceongoodsandservice','Currentaccountbalance_BoPcurrent','Nettradeingoodsandservices_BoPcu','Unemploymenttotal_pctoftotallabo','DGDP','RYPC','CGEB','PSBR','BINT','PUDP','SODD','CARA','IRTD','TDPX','TDPY','TSPY','INPS','INPY','XRRE']
test_predict=results_ols.predict(test[exog_var])
test_predict.describe()

count    496.000000
mean       0.010081
std        0.028134
min       -0.063619
25%       -0.006434
50%        0.006555
75%        0.021014
max        0.239043
dtype: float64

**What is the problem with the predicted value of default?**

Answer: Value of default should vary between one and zero. However, with this model the dependent variable takes on negative values of prediction term 'default_rr'. Also, with this model there exists strong multicolinearity

**2. Estimate a logit model for the training sample and use bootstrap to get the standard errors of coefficients.**

In [196]:
results_logit = logit(formula, train).fit()
print(results_logit.summary())

Optimization terminated successfully.
         Current function value: 0.079749
         Iterations 12
                           Logit Regression Results                           
Dep. Variable:             default_RR   No. Observations:                  773
Model:                          Logit   Df Residuals:                      760
Method:                           MLE   Df Model:                           12
Date:                Tue, 11 Feb 2020   Pseudo R-squ.:                  0.1668
Time:                        15:13:44   Log-Likelihood:                -61.646
converged:                       True   LL-Null:                       -73.987
Covariance Type:            nonrobust   LLR p-value:                   0.01641
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                           -3.2338      2.017     -1.6

**3. Calculate the marginal effect for Total International Reserves (“irtd”) based on the logit model.**

In [156]:
# marginal effect
results.get_margeff(at='mean', method='dydx', atexog='IRTD', dummy=False).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:             default_RR
Method:                          dydx
At:                              mean
====================================================================================================
                                      dy/dx    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Netforeignassets_currentLCU       1.897e-17   1.83e-17      1.035      0.301    -1.7e-17    5.49e-17
Inflationconsumerprices_annualpc -1.289e-05      0.000     -0.118      0.906      -0.000       0.000
Externalbalanceongoodsandservice     0.0002      0.000      0.711      0.477      -0.000       0.001
Currentaccountbalance_BoPcurrent  2.678e-13   3.77e-13      0.711      0.477    -4.7e-13    1.01e-12
Nettradeingoodsandservices_BoPcu -7.592e-14   3.09e-13     -0.246      0.806   -6.81e-13    5.29e-13
Unemploymenttotal_pctoftotallabo -5.661e-05      0.000     -0.209      0.834      -0.001       0.000
DGDP                                -0.0003      0.001     -0.354      0.723      -0.002       0.002
RYPC                                 0.0003      0.001      0.312      0.755      -0.001       0.002
CGEB                               7.96e-05      0.000      0.521      0.602      -0.000       0.000
PSBR                                -0.0001      0.000     -0.363      0.716      -0.001       0.000
BINT                             -1.835e-06   7.68e-05     -0.024      0.981      -0.000       0.000
PUDP                                 0.0001   8.26e-05      1.229      0.219   -6.04e-05       0.000
SODD                             -8.393e-06   6.89e-05     -0.122      0.903      -0.000       0.000
CARA                              3.268e-05      0.000      0.111      0.912      -0.001       0.001
IRTD                                -0.0002   8.04e-05     -2.560      0.010      -0.000   -4.83e-05
TDPX                             -8.064e-06   1.44e-05     -0.562      0.574   -3.62e-05    2.01e-05
TDPY                             -4.062e-05    7.7e-05     -0.528      0.598      -0.000       0.000
TSPY                                -0.0003      0.001     -0.432      0.666      -0.002       0.001
INPS                             -7.646e-05      0.000     -0.426      0.670      -0.000       0.000
INPY                                 0.0017      0.002      0.700      0.484      -0.003       0.007
XRRE                               1.44e-05   1.57e-05      0.915      0.360   -1.65e-05    4.53e-05
====================================================================================================
"""

**4. Estimate a probit model for the training sample and use bootstrap to get the standard errors of coefficients.**

In [0]:
def probit_wrap(formula, moddata):
    return probit(formula, moddata).fit()

In [0]:
# a function to run bootstraping with given number of iterations for given model

def bootstrap_with_model(func, nrep, formula, moddata):
    # func - input model function
    # nrep - number of iterations
    # formula - given formula for the model, required by statsmodels.formula.api
    # moddata - data used to run the model
    
    mod = func(formula, moddata)
    
    # store parameters of model without bootstrapping
    params0 = mod.params

    params0.name = 'Observed coef.'

    # run bootstrap with nrep number of iterations 
    # store parameters of each iteration as pd.Series in the list, params_list
    # params_list will be used to concatenate all pd.Series in a more efficient way by pd.concat than pd.append
    params_list = []

    for i in range(nrep):
        data_bs = resample(moddata, replace=True)

        temp_n = func(formula, data_bs).params

        params_list.append(temp_n)

        i+=1

    params = pd.concat(params_list, axis=1)

    # find standard erros
    errors = params.sub(params.mean(axis=1), axis=0)

    se = np.sqrt(((errors**2).sum(axis=1))/(nrep-1))

    se.name = 'Bootstrap Std. Err.'

    # find z score
    z = params0/se

    z.name = 'z'
    
    # find p-value
    p_values = scipy.stats.norm.sf(abs(z))*2

    p_values = pd.Series(np.round(p_values, 3), index=z.index, name='P>|z|')
    
    # find 95% confidence interval
    CI = scipy.stats.norm.interval(0.95, loc=params0, scale=se)

    CI_lower = pd.Series(CI[0], index=z.index, name='95% CI,lower')

    CI_high = pd.Series(CI[1], index=z.index, name='95% CI,high')
    
    # return:
    #    mod - the model run without bootstrapping, used for marginal effects later
    #    pd.concat returns the result table
    return mod, pd.concat([params0, se, z, p_values, CI_lower, CI_high], axis=1)

In [165]:
equation = 'default_RR ~ Netforeignassets_currentLCU + Inflationconsumerprices_annualpc + Externalbalanceongoodsandservice + Currentaccountbalance_BoPcurrent + Nettradeingoodsandservices_BoPcu + Unemploymenttotal_pctoftotallabo + DGDP + RYPC + CGEB + PSBR + BINT + PUDP + SODD + CARA + IRTD + TDPX + TDPY + TSPY + INPS + INPY + XRRE'
probit, result_probit = bootstrap_with_model(probit_wrap, 50, equation, train)

Optimization terminated successfully.
         Current function value: 0.079613
         Iterations 11
Optimization terminated successfully.
         Current function value: 0.064167
         Iterations 11
Optimization terminated successfully.
         Current function value: 0.048737
         Iterations 12
Optimization terminated successfully.
         Current function value: 0.067346
         Iterations 17
Optimization terminated successfully.
         Current function value: 0.050051
         Iterations 12
Optimization terminated successfully.
         Current function value: 0.049951
         Iterations 17
Optimization terminated successfully.
         Current function value: 0.051092
         Iterations 11
Optimization terminated successfully.
         Current function value: 0.066657
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.086574
         Iterations 13
Optimization terminated successfully.
         Current function value: 0.

In [200]:
result_probit

,Observed coef.,Bootstrap Std. Err.,z,P>|z|,"95% CI,lower","95% CI,high"
Intercept,-1.702901e+00,1.562216e+00,-1.090055,0.276,-4.764789e+00,1.358986e+00
Netforeignassets_currentLCU,1.855261e-15,5.318780e-13,0.003488,0.997,-1.040606e-12,1.044317e-12
Inflationconsumerprices_annualpc,-2.409873e-03,1.338533e-02,-0.180038,0.857,-2.864463e-02,2.382488e-02
Externalbalanceongoodsandservice,1.381201e-02,4.698523e-02,0.293965,0.769,-7.827736e-02,1.059014e-01
Currentaccountbalance_BoPcurrent,2.730080e-11,4.229807e-11,0.645439,0.519,-5.560189e-11,1.102035e-10
Nettradeingoodsandservices_BoPcu,-7.332647e-12,2.803184e-11,-0.261583,0.794,-6.227404e-11,4.760875e-11
Unemploymenttotal_pctoftotallabo,-9.455684e-04,3.914388e-02,-0.024156,0.981,-7.766617e-02,7.577504e-02
DGDP,-3.808581e-02,1.434412e-01,-0.265515,0.791,-3.192254e-01,2.430537e-01
RYPC,3.920344e-02,9.967055e-02,0.393330,0.694,-1.561472e-01,2.345541e-01
CGEB,8.276174e-03,3.268297e-02,0.253226,0.800,-5.578126e-02,7.233361e-02


**5. Calculate the marginal effect for Total International Reserves (“irtd”) based on the probit model.**

In [167]:
# marginal effect
probit.get_margeff(at='mean', method='dydx', atexog='IRTD', dummy=False).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:             default_RR
Method:                          dydx
At:                              mean
====================================================================================================
                                      dy/dx    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Netforeignassets_currentLCU       1.867e-17   2.17e-17      0.861      0.389   -2.39e-17    6.12e-17
Inflationconsumerprices_annualpc -2.425e-05      0.000     -0.230      0.818      -0.000       0.000
Externalbalanceongoodsandservice     0.0001      0.000      0.568      0.570      -0.000       0.001
Currentaccountbalance_BoPcurrent  2.747e-13   3.79e-13      0.725      0.469   -4.68e-13    1.02e-12
Nettradeingoodsandservices_BoPcu -7.378e-14   2.99e-13     -0.247      0.805    -6.6e-13    5.13e-13
Unemploymenttotal_pctoftotallabo -9.514e-06      0.000     -0.036      0.972      -0.001       0.001
DGDP                                -0.0004      0.001     -0.403      0.687      -0.002       0.001
RYPC                                 0.0004      0.001      0.437      0.662      -0.001       0.002
CGEB                              8.327e-05      0.000      0.478      0.633      -0.000       0.000
PSBR                             -3.461e-05      0.000     -0.106      0.916      -0.001       0.001
BINT                             -7.463e-07   7.79e-05     -0.010      0.992      -0.000       0.000
PUDP                              9.983e-05   9.27e-05      1.077      0.281   -8.18e-05       0.000
SODD                              2.116e-06   4.95e-05      0.043      0.966    -9.5e-05    9.92e-05
CARA                              9.323e-05      0.000      0.328      0.743      -0.000       0.001
IRTD                                -0.0002      0.000     -2.065      0.039      -0.000   -1.09e-05
TDPX                             -7.579e-06   1.34e-05     -0.564      0.573   -3.39e-05    1.88e-05
TDPY                             -3.964e-05   7.33e-05     -0.541      0.589      -0.000       0.000
TSPY                                -0.0003      0.001     -0.482      0.630      -0.002       0.001
INPS                                -0.0001      0.000     -0.542      0.588      -0.000       0.000
INPY                                 0.0020      0.003      0.731      0.465      -0.003       0.007
XRRE                              1.455e-05   1.78e-05      0.819      0.413   -2.03e-05    4.94e-05
====================================================================================================
"""

**6. A common way to assess the predictive efficiency of a binary model is the ROC curve. It plots the true positive rate ($=\frac{\text{True Positive}}{\text{True Positive+False Negative}}$) against the false positive rate ($=\frac{\text{False Positive}}{\text{False Positive+True Negative}}$). If a test is the diagonal line from bottom left corner to upper right corner, then the test is completely useless. The bigger the area is between the ROC curve and the diagonal line, the better the test (Read more about ROC curve in [https://en.wikipedia.org/wiki/Receiver_operating_characteristic]). Plot the ROC curve for logit and probit using the test sample and compare the accuracy of the two predictions.**

In [0]:
exog_var=['Netforeignassets_currentLCU','Inflationconsumerprices_annualpc','Externalbalanceongoodsandservice','Currentaccountbalance_BoPcurrent','Nettradeingoodsandservices_BoPcu','Unemploymenttotal_pctoftotallabo','DGDP','RYPC','CGEB','PSBR','BINT','PUDP','SODD','CARA','IRTD','TDPX','TDPY','TSPY','INPS','INPY','XRRE']
test_predict_logit=results_logit.predict(test[exog_var])
#test_predict_logit.describe()

Check the help file of roc_curve and roc_auc_score

In [0]:
# help(sklm.roc_curve)
# help(sklm.roc_auc_score)

**7. (extra points) Use machine learning methods to select variables from the list of all variables and then run a logit regression and plot the ROC curve. Does this improve the prediction?**



---

Submission by:
Mahesh Kumar
